In [ ]:
# importing the data

import pandas as pd
import os
from google.colab import files
uploaded = files.upload()

#uploaded_files = ["C:/Users/ising/Downloads/promptListBugs.xlsx - Sheet1.csv", "C:/Users/ising/Downloads/promptListMammal.xlsx - Sheet1.csv", "C:/Users/ising/Downloads/promptListBird.xlsx - Sheet1.csv", "C:/Users/ising/Downloads/promptListFish.xlsx - Sheet1.csv", "C:/Users/ising/Downloads/promptListReptile.xlsx - Sheet1.csv"]


Saving promptListReptile.xlsx - Sheet1.csv to promptListReptile.xlsx - Sheet1.csv
Saving promptListMammal.xlsx - Sheet1.csv to promptListMammal.xlsx - Sheet1.csv
Saving promptListFish.xlsx - Sheet1.csv to promptListFish.xlsx - Sheet1.csv
Saving promptListBugs.xlsx - Sheet1.csv to promptListBugs.xlsx - Sheet1.csv
Saving promptListBird.xlsx - Sheet1.csv to promptListBird.xlsx - Sheet1.csv


In [ ]:
os.rename('promptListReptile.xlsx - Sheet1.csv', 'Reptiles.csv')
os.rename('promptListMammal.xlsx - Sheet1.csv', 'Mammals.csv')
os.rename('promptListFish.xlsx - Sheet1.csv', 'Fish.csv')
os.rename('promptListBugs.xlsx - Sheet1.csv', 'Bugs.csv')
os.rename('promptListBird.xlsx - Sheet1.csv', 'Birds.csv')

In [ ]:
reptiles = pd.read_csv('Reptiles.csv')
mammals = pd.read_csv('Mammals.csv')
fish = pd.read_csv('Fish.csv')
bugs = pd.read_csv('Bugs.csv')
birds = pd.read_csv('Birds.csv')

In [ ]:
print(reptiles.head())

                                          0
0                Is a Angelshark a reptile?
1          Is a Pig-nosed Turtle a reptile?
2  Is a Chinese Giant Salamander a reptile?
3              Is a Pillar Coral a reptile?
4    Is a Pygmy Three-toed Sloth a reptile?


## Picking random control vectors

In [ ]:
import random

control_reptile = random.choice(reptiles['0'])
control_mammal = random.choice(mammals['0'])
control_fish = random.choice(fish['0'])
control_bug = random.choice(bugs['0'])
control_bird = random.choice(birds['0'])
print(control_reptile)
print(control_mammal)
print(control_fish)
print(control_bug)
print(control_bird)


Is a Morne Dubois Least Gecko a reptile?
Is a Australian Mist a mammal?
Is a Wrymouth a fish?
Is a alfalfa webworm a bug?
Is a carib a bird?


## Picking 15 other observations randomly (excluding the random control vector)

In [ ]:
rep_ex = reptiles['0'].loc[reptiles['0'] != control_reptile]
mam_ex = reptiles['0'].loc[reptiles['0'] != control_mammal]
fish_ex = reptiles['0'].loc[reptiles['0'] != control_fish]
bugs_ex = reptiles['0'].loc[reptiles['0'] != control_bug]
birds_ex = reptiles['0'].loc[reptiles['0'] != control_bird]

reptiles_15 = random.sample(rep_ex.tolist(), 15)
mammals_15 = random.sample(mam_ex.tolist(), 15)
fish_15 = random.sample(fish_ex.tolist(), 15)
bugs_15 = random.sample(bugs_ex.tolist(), 15)
birds_15 = random.sample(birds_ex.tolist(), 15)

## Vector Embeddings for the 5 Control Vectors

In [ ]:
from transformers import BertModel, BertTokenizer
import torch

model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

control_prompts = {
    'reptile': control_reptile,
    'mammal': control_mammal,
    'fish': control_fish,
    'bug': control_bug,
    'bird': control_bird
}

control_embeddings = {}

# loop over each control prompt
for category, prompt in control_prompts.items():

    tokens = tokenizer(prompt, return_tensors='pt')
    outputs = model(**tokens)

    last_hidden_states = outputs.last_hidden_state

    sentence_embedding = torch.mean(last_hidden_states, dim=1)

    control_embeddings[category] = sentence_embedding.detach().numpy()



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

## Vector embeddings for each random sample of 15 for each animal class type

In [ ]:
def get_embeddings(texts):
    embeddings = []
    for text in texts:
        # Tokenize and convert to tensor
        tokens = tokenizer(text, return_tensors='pt')
        outputs = model(**tokens)

        # Extract the embeddings from the last layer
        last_hidden_states = outputs.last_hidden_state

        # Take the mean of the last layer hidden states along the token dimension to get a single vector for the entire input
        sentence_embedding = torch.mean(last_hidden_states, dim=1)

        # Convert the PyTorch tensor to a NumPy array and append to the list of embeddings
        embeddings.append(sentence_embedding.detach().numpy())
    return embeddings

# Define a dictionary to store embeddings for each category of random samples
random_sample_embeddings = {
    'reptiles': get_embeddings(reptiles_15),
    'mammals': get_embeddings(mammals_15),
    'fish': get_embeddings(fish_15),
    'bugs': get_embeddings(bugs_15),
    'birds': get_embeddings(birds_15)
}

## Creating the datasets

In [ ]:
# Define observation names
rep_obs = [f"R{i}" for i in range(1, 16)]
mam_obs = [f"M{i}" for i in range(1, 16)]
fish_obs = [f"F{i}" for i in range(1, 16)]
bugs_obs = [f"Bu{i}" for i in range(1, 16)]
birds_obs = [f"Bi{i}" for i in range(1, 16)]

# Define control vector names
control_vector_names = ["Reptile", "Mammal", "Fish",  "Bug", "Bird"]

# Create empty DataFrames for each class
reptiles_df = pd.DataFrame(index=rep_obs, columns=control_vector_names)
mammals_df = pd.DataFrame(index=mam_obs, columns=control_vector_names)
fish_df = pd.DataFrame(index=fish_obs, columns=control_vector_names)
bugs_df = pd.DataFrame(index=bugs_obs, columns=control_vector_names)
birds_df = pd.DataFrame(index=birds_obs, columns=control_vector_names)

reptiles_df


,Reptile,Mammal,Fish,Bug,Bird
R1,NaN,NaN,NaN,NaN,NaN
R2,NaN,NaN,NaN,NaN,NaN
R3,NaN,NaN,NaN,NaN,NaN
R4,NaN,NaN,NaN,NaN,NaN
R5,NaN,NaN,NaN,NaN,NaN
R6,NaN,NaN,NaN,NaN,NaN
R7,NaN,NaN,NaN,NaN,NaN
R8,NaN,NaN,NaN,NaN,NaN
R9,NaN,NaN,NaN,NaN,NaN
R10,NaN,NaN,NaN,NaN,NaN


In [ ]:
import numpy as np
from scipy.spatial import distance
# Function to calculate Euclidean distance between two embeddings
def cosine_distance(embedding1, embedding2):
    for row in embedding1:
      for s in embedding2:
        d = distance.cosine(row, s)
    return d

# Loop over each row and column to populate the dataframe
for i, row_label in enumerate(reptiles_df.index):
    for j, col_label in enumerate(reptiles_df.columns):
        # Get the embeddings for the corresponding row and column labels
        row_embedding = random_sample_embeddings['reptiles'][i]
        col_embedding = control_embeddings[col_label.lower()]

        # Calculate Euclidean distance and store in the dataframe
        reptiles_df.at[row_label, col_label] = cosine_distance(row_embedding, col_embedding)

# Highlight the minimum value in each row
reptiles_df_highlighted = reptiles_df.style.apply(lambda row: ['background-color: yellow' if cell == row.min() else '' for cell in row], axis=1)

# Display the highlighted dataframe for reptiles
print("Reptiles DataFrame:")
display(reptiles_df_highlighted)



Reptiles DataFrame:


,Reptile,Mammal,Fish,Bug,Bird
R1,0.141907,0.207229,0.229015,0.232566,0.234200
R2,0.139678,0.236665,0.283793,0.278841,0.297284
R3,0.127814,0.233041,0.299215,0.248332,0.294119
R4,0.121236,0.141275,0.196514,0.207448,0.173240
R5,0.109846,0.151942,0.219182,0.213423,0.189246
R6,0.162610,0.232188,0.308105,0.287057,0.275488
R7,0.064887,0.243604,0.282700,0.260976,0.286250
R8,0.211493,0.204575,0.285194,0.286931,0.225838
R9,0.133935,0.135135,0.187041,0.210462,0.174341
R10,0.049876,0.179645,0.240086,0.226761,0.240075


In [ ]:
# Loop over each row and column to populate the dataframe
for i, row_label in enumerate(mammals_df.index):
    for j, col_label in enumerate(mammals_df.columns):
        # Get the embeddings for the corresponding row and column labels
        row_embedding = random_sample_embeddings['mammals'][i]
        col_embedding = control_embeddings[col_label.lower()]

        # Calculate Euclidean distance and store in the dataframe
        mammals_df.at[row_label, col_label] = cosine_distance(row_embedding, col_embedding)

# Highlight the minimum value in each row
mammals_df_highlighted = mammals_df.style.apply(lambda row: ['background-color: yellow' if cell == row.min() else '' for cell in row], axis=1)

# Display the highlighted dataframe for mammals
print("Mammals DataFrame:")
display(mammals_df_highlighted)


Mammals DataFrame:


,Reptile,Mammal,Fish,Bug,Bird
M1,0.075475,0.260490,0.291524,0.278718,0.319730
M2,0.050033,0.147928,0.201269,0.203248,0.194902
M3,0.154140,0.283916,0.277804,0.277198,0.310001
M4,0.085464,0.206268,0.295562,0.297446,0.241007
M5,0.168266,0.266536,0.334115,0.251614,0.336316
M6,0.163339,0.211125,0.290744,0.247286,0.255292
M7,0.147492,0.235000,0.283512,0.236852,0.290068
M8,0.074767,0.188471,0.246895,0.225532,0.234158
M9,0.109846,0.151942,0.219182,0.213423,0.189246
M10,0.168457,0.201910,0.283544,0.280041,0.211365


In [ ]:
# Loop over each row and column to populate the dataframe
for i, row_label in enumerate(fish_df.index):
    for j, col_label in enumerate(fish_df.columns):
        # Get the embeddings for the corresponding row and column labels
        row_embedding = random_sample_embeddings['fish'][i]
        col_embedding = control_embeddings[col_label.lower()]

        # Calculate Euclidean distance and store in the dataframe
        fish_df.at[row_label, col_label] = cosine_distance(row_embedding, col_embedding)

# Highlight the minimum value in each row
fish_df_highlighted = fish_df.style.apply(lambda row: ['background-color: yellow' if cell == row.min() else '' for cell in row], axis=1)

# Display the highlighted dataframe for fish
print("Fish DataFrame:")
display(fish_df_highlighted)

Fish DataFrame:


,Reptile,Mammal,Fish,Bug,Bird
F1,0.080319,0.141610,0.217604,0.238595,0.178202
F2,0.147492,0.235000,0.283512,0.236852,0.290068
F3,0.131648,0.238002,0.243678,0.248444,0.270690
F4,0.052278,0.161212,0.226966,0.250691,0.212759
F5,0.093513,0.200448,0.264631,0.259036,0.202799
F6,0.112407,0.215612,0.287631,0.229426,0.257626
F7,0.231037,0.169398,0.176990,0.238685,0.182996
F8,0.158338,0.174595,0.217740,0.173965,0.223877
F9,0.137060,0.117935,0.169840,0.196943,0.148794
F10,0.064887,0.243604,0.282700,0.260976,0.286250


In [ ]:
# Loop over each row and column to populate the dataframe
for i, row_label in enumerate(bugs_df.index):
    for j, col_label in enumerate(bugs_df.columns):
        # Get the embeddings for the corresponding row and column labels
        row_embedding = random_sample_embeddings['bugs'][i]
        col_embedding = control_embeddings[col_label.lower()]

        # Calculate Euclidean distance and store in the dataframe
        bugs_df.at[row_label, col_label] = cosine_distance(row_embedding, col_embedding)

# Highlight the minimum value in each row
bugs_df_highlighted = bugs_df.style.apply(lambda row: ['background-color: yellow' if cell == row.min() else '' for cell in row], axis=1)

# Display the highlighted dataframe for fish
print("Bugs DataFrame:")
display(bugs_df_highlighted)

Bugs DataFrame:


,Reptile,Mammal,Fish,Bug,Bird
Bu1,0.170090,0.130569,0.173115,0.199139,0.189463
Bu2,0.116414,0.163847,0.183546,0.206127,0.188442
Bu3,0.131648,0.238002,0.243678,0.248444,0.270690
Bu4,0.049876,0.179645,0.240086,0.226761,0.240075
Bu5,0.200314,0.183011,0.199963,0.259384,0.175774
Bu6,0.114488,0.132081,0.199438,0.214467,0.180451
Bu7,0.159281,0.192236,0.268370,0.240939,0.215248
Bu8,0.133397,0.183021,0.216993,0.219128,0.247482
Bu9,0.123540,0.196936,0.236389,0.236269,0.224182
Bu10,0.211493,0.204575,0.285194,0.286931,0.225838


In [ ]:
# Loop over each row and column to populate the dataframe
for i, row_label in enumerate(birds_df.index):
    for j, col_label in enumerate(birds_df.columns):
        # Get the embeddings for the corresponding row and column labels
        row_embedding = random_sample_embeddings['birds'][i]
        col_embedding = control_embeddings[col_label.lower()]

        # Calculate Euclidean distance and store in the dataframe
        birds_df.at[row_label, col_label] = cosine_distance(row_embedding, col_embedding)

# Highlight the minimum value in each row
birds_df_highlighted = birds_df.style.apply(lambda row: ['background-color: yellow' if cell == row.min() else '' for cell in row], axis=1)

# Display the highlighted dataframe for fish
print("Birds DataFrame:")
display(birds_df_highlighted)

Birds DataFrame:


,Reptile,Mammal,Fish,Bug,Bird
Bi1,0.045447,0.200131,0.226575,0.225072,0.251904
Bi2,0.105127,0.157608,0.214173,0.227072,0.223849
Bi3,0.112407,0.215612,0.287631,0.229426,0.257626
Bi4,0.076057,0.267077,0.270812,0.291125,0.322164
Bi5,0.093513,0.200448,0.264631,0.259036,0.202799
Bi6,0.109846,0.151942,0.219182,0.213423,0.189246
Bi7,0.141305,0.098419,0.156267,0.190050,0.148769
Bi8,0.100785,0.167849,0.211362,0.195442,0.231284
Bi9,0.137060,0.117935,0.169840,0.196943,0.148794
Bi10,0.118370,0.174795,0.212944,0.222586,0.196766
